In [1]:
from Twitter_keeper import PullTweetsData

puller = PullTweetsData()

test_cases = [
            ("Hello! World🌍 https://www.example.com", "Hello/World"),
            ("Hello 😀 World 🚀🚀https://www.example.com", "Hello/World"),
            ("#ทรงอย่างแบด ลูกสาวชอบมากถึงขนาดให้เก็บซองเลย์ที่เค้านั่งกินระหว่างรอพี่ๆpaper planesมา ไม่รู้ว่าอีกหน่อยเค้าจะลืมมั้ย แต่ก็ทำให้ตามที่เค้าขอ เพราะความสุขของแต่ละคนไม่เหมือนกัน ถ้าศิลปินคือแรงบันดาลใจให้ลูกเรากลายเป็นเด็กดีคนดี แม่อย่างเราก็ต้องขอบคุณเค้ามากๆนะ https://t.co/oWconpN1iP", "Hello/World"),
        ]
for text, expected_output in test_cases:
    print(puller.preprocessText(text))

Hello/World
Hello/World
แบด/ลูกสาว/ชอบ/ถึงขนาด/ซอง/์/เค้า/นั่ง/กิน/รอ/พี่/paper/planes/รู้/อีกหน่อย/เค้า/ลืม/เค้า/ความสุข/แต่ละคน/เหมือนกัน/ศิลปิน/แรงบันดาลใจ/ลูก/กลายเป็น/เด็กดี/คนดี/แม่/ขอบคุณ/เค้า
